<a href="https://colab.research.google.com/github/tracedence/Data-analysis/blob/main/sms_spam_classifier_(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip

--2022-07-07 13:09:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   790KB/s    in 0.3s    

2022-07-07 13:09:59 (790 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [13]:
!unzip smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [51]:
df = pd.read_csv("./SMSSpamCollection", sep="\t", names=["label", "messge"])
df.head()

,label,messge
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [56]:
df.shape

(5572, 3)

## checking balanced dataset or not

In [97]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### it's an imbalanced datasets

Data cleaning

In [54]:
# removing symbols
# lowercase
# steming
# remove stop words
lemmatizer = WordNetLemmatizer()
df["clean_messge"] = df["messge"].apply(lambda sentence: " ".join(word for word in sentence.split() if word not in set(stopwords.words("english")) ))
df["clean_messge"] = df["clean_messge"].apply(lambda x: (re.sub("[^a-zA-Z]", " ", x.lower())))
df["clean_messge"] = df["clean_messge"].apply(lambda sentence: " ".join([lemmatizer.lemmatize(word) for word in sentence.split()]))


In [55]:
df.head()

,label,messge,clean_messge
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i think go usf life around though


In [88]:
## creating bag of words or vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()
message_array = tf_idf.fit_transform(list(df["clean_messge"])).toarray()

In [89]:
message_array.shape

(5572, 7209)

In [90]:
y = pd.get_dummies(df["label"])
y = y.iloc[:, 1].values

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_array, y, test_size=0.3, random_state=40)
X_train.shape

(3900, 7209)

In [92]:
X_test.shape

(1672, 7209)

In [93]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB().fit(X_train, y_train)

In [94]:
y_pred = nb_classifier.predict(X_test)

In [95]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[1462,    0],
       [  58,  152]])

In [96]:
accuracy_score(y_test, y_pred)

0.965311004784689